In [13]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import folium
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics import silhouette_score



df = pd.read_csv("KIC.csv")

df.head()

,번호,센터명,주소,클러스터,위도,경도,은행,편의점,지하철 개수,지하철 점수,지하철등급,국도(km),고속도로(km),도로점수,도로등급
0,0,강릉 공공임대형 지식산업센터,강원특별자치도 강릉시 사천면 방동리 805-3,NaN,37.806207,128.854086,E,E,0,0.0,E,0.91,7.81,1.63,A
1,1,동해지식산업센터,강원특별자치도 동해시 구호동 216,NaN,37.481576,129.133811,C,E,0,0.0,E,3.65,6.42,4.65,E
2,2,표준공장1동,강원특별자치도 동해시 구호동 247번지,NaN,37.480842,129.153095,E,E,0,0.0,E,5.65,9.70,7.14,E
3,3,표준공장2동,강원특별자치도 동해시 구호동 247번지,NaN,37.480842,129.153095,E,E,0,0.0,E,5.65,9.70,7.14,E
4,4,표준공장3동,강원특별자치도 동해시 구호동 247번지,NaN,37.480842,129.153095,E,E,0,0.0,E,5.65,9.70,7.14,E


In [14]:
coords = df[['위도', '경도']].dropna().values

# 표준화
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)

results = []

for n_clusters in range(100, 301):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(coords_scaled)
    inertia = kmeans.inertia_
    # 실루엣 점수는 군집이 1개일 때 계산 불가, 2개 이상일 때만 계산
    if len(set(labels)) > 1 and len(set(labels)) < len(coords_scaled):
        silhouette = silhouette_score(coords_scaled, labels)
    else:
        silhouette = np.nan
    results.append({
        'n_clusters': n_clusters,
        'inertia': inertia,
        'silhouette_score': silhouette
    })

# 결과를 데이터프레임으로 변환 및 출력
results_df = pd.DataFrame(results)
print(results_df)

     n_clusters   inertia  silhouette_score
0           100  1.162689          0.668513
1           101  1.113572          0.668456
2           102  1.111509          0.666136
3           103  1.106450          0.665366
4           104  1.087578          0.664407
..          ...       ...               ...
196         296  0.047720          0.590540
197         297  0.047282          0.589992
198         298  0.046841          0.589962
199         299  0.046386          0.590139
200         300  0.045993          0.589920

[201 rows x 3 columns]


100~110개가 좋다는결론이 나오긴함 근데음..

-----------

In [20]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import folium
import matplotlib.pyplot as plt
import matplotlib

# 데이터 로드
df = pd.read_csv("KIC.csv")

# 위도/경도 데이터 추출 및 결측치 제거
coords = df[['위도', '경도']].dropna().values

# 표준화
scaler = StandardScaler()
coords_scaled = scaler.fit_transform(coords)

# KMeans 클러스터링 (클러스터 개수는 데이터에 맞게 조정)
n_clusters = 180
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(coords_scaled)

# 클러스터 결과를 원본 데이터프레임에 반영 (결측치 행은 -1로 처리)
df['클러스터'] = -1
df.loc[df[['위도', '경도']].dropna().index, '클러스터'] = labels

# 결과 확인
print(df['클러스터'].value_counts())
print(df.head())

# 결과 저장
df.to_csv("KIC_clustered_kmeans.csv", index=False, encoding='utf-8-sig')

# 지도 시각화 (folium)
center_lat = df['위도'].mean()
center_lon = df['경도'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=8)

n_clusters_real = len(set(labels))
colors = plt.cm.tab20(np.linspace(0, 1, max(n_clusters_real, 1)))

for idx, row in df.iterrows():
    if np.isnan(row['위도']) or np.isnan(row['경도']):
        continue
    cluster = int(row['클러스터'])
    if cluster == -1:
        color = 'gray'
    else:
        color = matplotlib.colors.rgb2hex(colors[cluster % len(colors)])
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=4,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"센터명: {row['센터명']}<br>클러스터: {cluster}"
    ).add_to(m)

m.save("KIC_cluster_map_kmeans.html")
print("KMeans 클러스터링 결과를 KIC_clustered_kmeans.csv로 저장하고, 지도는 KIC_cluster_map_kmeans.html로 저장했습니다.")

클러스터
109    77
17     62
35     49
151    47
127    40
       ..
165     1
175     1
173     1
178     1
56      1
Name: count, Length: 180, dtype: int64
   번호              센터명                         주소  클러스터         위도  \
0   0  강릉 공공임대형 지식산업센터  강원특별자치도 강릉시 사천면 방동리 805-3    11  37.806207   
1   1         동해지식산업센터        강원특별자치도 동해시 구호동 216    30  37.481576   
2   2           표준공장1동      강원특별자치도 동해시 구호동 247번지    30  37.480842   
3   3           표준공장2동      강원특별자치도 동해시 구호동 247번지    30  37.480842   
4   4           표준공장3동      강원특별자치도 동해시 구호동 247번지    30  37.480842   

           경도 은행 편의점  지하철 개수  지하철 점수 지하철등급  국도(km)  고속도로(km)  도로점수 도로등급  
0  128.854086  E   E       0     0.0     E    0.91      7.81  1.63    A  
1  129.133811  C   E       0     0.0     E    3.65      6.42  4.65    E  
2  129.153095  E   E       0     0.0     E    5.65      9.70  7.14    E  
3  129.153095  E   E       0     0.0     E    5.65      9.70  7.14    E  
4  129.153095  E   E       0     0.0     E    5.65     